## Setup


*   Perform imports and installs
*   Retrieve OpenAI API key
*   Define JSON format for output of translation text completion



In [ ]:
!pip install openai

In [ ]:
import openai, textwrap, base64, io, json, shutil

from openai import OpenAI
from pydantic import BaseModel
from IPython.display import Audio, display
from pathlib import Path
from datetime import datetime

from google.colab import drive
drive.mount('/content/gdrive')

from google.colab import userdata
openai.api_key = userdata.get("OPENAI_API_KEY")

In [ ]:
import os

output_folder_text = Path("/content/gdrive/Shareddrives/LL_CENTRAL/LL_PROJECTS/24.1/COLAB/ll_color_colabs/_ai_lab/utilities/output_folder/text")
output_folder_audio = Path("/content/gdrive/Shareddrives/LL_CENTRAL/LL_PROJECTS/24.1/COLAB/ll_color_colabs/_ai_lab/utilities/output_folder/audio")


class output(BaseModel):
  text: str

## Translation
* define translation function
* define variables containing each author's text
* run function and print results

In [ ]:
client = OpenAI(api_key=openai.api_key)

_stamp = lambda: datetime.now().strftime("%Y%m%d_%H%M%S")

def translate(target_lang: str,
                   source_text: str,
                   system_prompt: str = "",
                   model: str = "gpt-4o-2024-08-06",
                   wrap: int = 80) -> str:
    """Translate text and save .txt to output_folder."""
    resp = client.responses.create(
        model=model,
        input=[
            {"role": "system", "content": system_prompt},
            {"role": "user",
             "content": f"Translate the following text into {target_lang}:\n\n{source_text}"},
        ],
    )
    translated = resp.output[0].content[0].text.strip()

    file_name = f"{_stamp()}_{target_lang}.txt"
    text_file_path = output_folder_text / file_name
    text_file_path.write_text(
        textwrap.fill(translated, width=wrap), encoding="utf-8"
    )
    print(f"Saved translation → {text_file_path}")

    return translated


def speak_the_text(tts_text: str,
                   model: str = "tts-1",
                   voice: str = "alloy") -> None:
    """Stream TTS to MP3 in output_folder_audio and autoplay."""
    local_file_path = Path(f"/tmp/speech_{_stamp()}.mp3")
    drive_file_path = output_folder_audio / local_file_path.name

    with client.audio.speech.with_streaming_response.create(
        model=model,
        voice=voice,
        input=tts_text
    ) as resp:
        resp.stream_to_file(local_file_path)

    if local_file_path.exists():
        shutil.copy(local_file_path, drive_file_path)
        display(Audio(str(drive_file_path), autoplay=True))
        print(f"Audio saved → {drive_file_path}")
        local_file_path.unlink() # Clean up the temporary file

In [ ]:
hyungdo = """엄마 걱정
기형도
열무 삼십 단을 이고
시장에 간 우리 엄마
안 오시네, 해는 시든 지 오래
나는 찬밥처럼 방에 담겨
아무리 천천히 숙제를 해도
엄마 안 오시네, 배춧잎 같은 발소리 타박타박
안 들리네, 어둡고 무서워
금간 창 틈으로 고요히 빗소리
빈 방에 혼자 엎드려 훌쩍거리던
아주 먼 옛날
지금도 내 눈시울을 뜨겁게 하는
그 시절, 내 유년의 윗목"""

wittgenstein = """
Tractatus 1–1.13
1
Die Welt ist alles, was der Fall ist.
1.1
Die Welt ist die Gesamtheit der Tatsachen, nicht der Dinge.
1.11
Die Welt ist durch die Tatsachen bestimmt und dadurch, dass es alle Tatsachen sind.
1.12
Denn, die Gesamtheit der Tatsachen bestimmt, was der Fall ist und auch, was alles nicht der Fall ist.
1.13
Die Tatsachen im logischen Raum sind die Welt.
"""

leopardi = """
Giacomo Leopardi, "Alla luna" (Canti 14)

O graziosa luna, io mi rammento
Che, or volge l’anno, sovra questo colle
Io venia pien d’angoscia a rimirarti:
E tu pendevi allor su quella selva
Siccome or fai, che tutta la rischiari.
Ma nebuloso e tremulo dal pianto
Che mi sorgea sul ciglio, alle mie luci
Il tuo volto apparia, che travagliosa
Era mia vita: ed è, nè cangia stile,
O mia diletta luna. E pur mi giova
La ricordanza, e il noverar l’etate
Del mio dolore. Oh come grato occorre
Nel tempo giovanil, quando ancor lungo
La speme e breve ha la memoria il corso,
Il rimembrar delle passate cose,
Ancor che triste, e che l’affanno duri!

"""

confucius = """
子曰：「學而時習之，不亦說乎？有朋自遠方來，不亦樂乎？人不知而不慍，不亦君子乎？」
"""

In [ ]:
english_wittgenstein = translate(
    "English", wittgenstein,
    "You can speak every language as a native perfectly."
)
print("\n— Wittgenstein → English —\n", english_wittgenstein)

english_hyungdo = translate(
    "English", hyungdo,
    "You can speak every language as a native perfectly."
)
print("\n— Hyungdo → English —\n", english_hyungdo)
speak_the_text(english_hyungdo)

german_leopardi = translate(
    "German", leopardi,
    "You can speak every language as a native perfectly."
)
print("\n— Leopardi → German —\n", german_leopardi)
speak_the_text(german_leopardi)